In [32]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'sk-8N6IjjnAipJKCSGw0aD0E540E5Bb4e8c86D3727cC7F5EdC3'

from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = 'sk-rJHGzAs4C0kSriu83125237077814d84Be4aCf56F0F82d7d'
# 配置模型地址
os.environ["OPENAI_API_BASE"] = "https://api.gpts.vin/v1"
# 配置模型版本
model = ChatOpenAI(model="gpt-3.5-turbo")

In [33]:
# 构建提示模板
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
) 
chain = prompt | model
response = chain.invoke({"messages": [HumanMessage(content="hi,i am bob")]})
response.content

'Hello Bob! How can I assist you today?'

In [34]:
# 构建消息历史
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)
config = {'configurable': {"session_id": "abc5"}}
response = with_message_history.invoke([HumanMessage(content="hi,i am jim")],config)
response.content

Parent run 9dc1be66-4b8f-457f-93fa-a5c912b8f88f not found for run 01a6ede5-3c7c-4485-a796-5b37ae957961. Treating as a root run.


'Hello Jim! How can I assist you today?'

In [35]:
# 通过config使用历史消息
response = with_message_history.invoke([HumanMessage(content="what is my name?")],config)
response.content

Parent run c19f69d7-66b8-4a47-ad5c-6a173959feaa not found for run a604721d-de79-456b-89ff-123400e2914d. Treating as a root run.


'Your name is Jim.'

In [36]:
# 构建提示模版，带有变量language
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [37]:
# 按照中文回答问题
chain.invoke({"messages": [HumanMessage(content="hi,i am bob")],"language":"chinese"}).content

'你好，Bob！有什么我可以帮到你的吗？'

In [38]:
# 构建历史消息
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
# 创建模版
config = {'configurable': {"session_id": "abc11"}}
# 带有历史消息的回答
with_message_history.invoke(
    {'messages':[HumanMessage(content="hi,i am todd")],"language":"chinese"}
    ,config
)
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="what is my name?")],"language":"chinese"}
    ,config
)
response.content


Parent run 267138fd-d0e8-4c9a-bcd5-0532239a1a9a not found for run 80043ff9-c1b0-4b39-946c-6511252d0167. Treating as a root run.
Parent run 4c444da0-ca71-4362-ab73-14bcb6bb8862 not found for run d82b1350-e4e5-4da0-868d-e7901855465b. Treating as a root run.


'你的名字是Todd。'